In [62]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import matplotlib.pyplot as plt
from math import sin, cos, sqrt, atan2, radians

In [63]:
# import data
links = pd.read_csv("links.csv")
nodes = pd.read_csv("nodes.csv")
loads = pd.read_csv("loads2.csv")

In [64]:
links = links.groupby(['Node1', 'Node2']).sum()

# parameters
loss = 0.05 / 100  # loss per mile (2% per 100mi)
T = 42370  # TIME STEP JAN 1 2010
solar = 0.5  # amount of solar available

# time steps
TIMES = loads['Time'][:24].values



In [65]:
def dist(lat1in, lat2in, lon1in, lon2in):
    # approximate radius of earth in miles
    R = 3958.8

    lat1 = radians(lat1in)
    lon1 = radians(lon1in)
    lat2 = radians(lat2in)
    lon2 = radians(lon2in)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c


In [66]:
# create model
m = gp.Model("test")
m.setParam('OutputFlag', 0)

# create cost function
cost = gp.LinExpr(0)

# dictionary of junction (net power) equations at each node
j = {}

# create a linear expression at each node
for i in nodes.index:
    node = nodes['Node'][i]
    j[node] = gp.LinExpr(0)


In [67]:
# FLOWS dictionary indexed by tuple (from, to) pairs
f = {}

# distance dictionary indexed by tuple
d = {}

# create flow variables
for i in links.index:
    # index nodes
    n1 = i[0]
    n2 = i[1]

    # create flow (decision) variable in each direction
    f[(n1, n2)] = m.addVar(lb=0, ub=links['Limit'][i])
    f[(n2, n1)] = m.addVar(lb=0, ub=links['Limit'][i])

    # FIND DISTANCE
    lat1 = nodes.loc[nodes['Node'] == n1, 'Latitude'].values[0]
    lat2 = nodes.loc[nodes['Node'] == n2, 'Latitude'].values[0]
    long1 = nodes.loc[nodes['Node'] == n1, 'Longitude'].values[0]
    long2 = nodes.loc[nodes['Node'] == n2, 'Longitude'].values[0]
    distance = dist(lat1, lat2, long1, long2)

    d[(n1, n2)] = distance
    d[(n2, n1)] = distance

    # add costs to objective
    cost.add(f[(n1, n2)], loss * distance)
    cost.add(f[(n2, n1)], loss * distance)

    # add to junctions (net power balance)
    j[n1].add(f[(n1, n2)], -1)
    j[n1].add(f[(n2, n1)], (1 - loss * distance))
    j[n2].add(f[(n1, n2)], (1 - loss * distance))
    j[n2].add(f[(n2, n1)], -1)

J_CONST = j

In [68]:

# filter to generating nodes
gens = nodes[nodes['Limit'] > 0]

# dictionary of generator decision variables
g = {}

for i in gens.index:
    # add the decision variable
    node = gens['Node'][i]
    lim = gens['Limit'][i]

    # scale solar
    if gens['Type'][i] == 'Solar':
        lim *= solar

    g[node] = m.addVar(lb=0, ub=lim)

    # add generation to net power at node
    j[node].add(g[node])

    # add cost to objective
    if gens['Type'][i] == 'Natural Gas':
        cost.add(g[node])

    if gens['Type'][i] == 'Coal':
        cost.add(g[node])


In [69]:
# LOADS
t_loads = loads.loc[loads['Time'] == T]
t_loads = t_loads.drop(columns=['Time', 'Net'])

# add to junction sets
for col in t_loads:
    j[int(col)].add(-1 * t_loads[col].values[0])

In [70]:
# Constraints
c = {}

for i in nodes.index:
    node = nodes['Node'][i]
    c[node] = m.addConstr(j[node] == 0)

# Objective
obj = m.setObjective(cost, GRB.MINIMIZE)

# Optimize
m.optimize()

In [71]:
# Generation Metrics
usage = {'Coal': 0, 'Natural Gas': 0, 'Nuclear': 0, 'Hydro': 0, 'Wind': 0, 'Solar': 0}
limit = {'Coal': 0, 'Natural Gas': 0, 'Nuclear': 0, 'Hydro': 0, 'Wind': 0, 'Solar': 0}

for i in gens.index:
    node = gens['Node'][i]
    usage[gens['Type'][i]] += g[node].getAttr("x")
    limit[gens['Type'][i]] += g[node].getAttr("UB")

tot_gen = sum(usage[i] for i in usage)

print("\n-----\n")
print("Generation")
for i in usage:
    print("%s: \t %.2f of total (%0.2f of capacity)" % (i, usage[i] / tot_gen, usage[i] / limit[i]))

# Flow Metrics
# Total Efficiency
print("\nTransmission")
print("Overall Efficiency: %.2f" % (loads.loc[loads['Time'] == T]['Net'].values[0] / tot_gen))

# Average Distance Traveled
powdist = sum((f[i].getAttr("x") * d[i]) for i in f)
print("Average Distance: %.2f" % (powdist / tot_gen))

# Average Utilization
# Percent of Binding Links


-----

Generation
Coal: 	 0.05 of total (0.12 of capacity)
Natural Gas: 	 0.43 of total (0.28 of capacity)
Nuclear: 	 0.14 of total (1.00 of capacity)
Hydro: 	 0.03 of total (1.00 of capacity)
Wind: 	 0.34 of total (0.99 of capacity)
Solar: 	 0.01 of total (1.00 of capacity)

Transmission
Overall Efficiency: 0.96
Average Distance: 81.19
